# Clustering

## 1. keyword 정제
    1) 주요 태그 추출
    - description 내용 포함 태그
    - description과 일치하는 단어가 없는 태그는 태그 그대로 사용
    - 태그 없는 187개 스튜디오는 description에서 추출한 단어 사용
     
     2) 유사 단어 묶어주기
    - WordCloudVec으로 단어간 유사도 판별

    >> 타겟 스튜디오와 동일한 태그 개수 세기

## 2. 범주형 데이터 Gower로 수치형 데이터로 변환
    1) 범주형 데이터는 category와 

In [4]:
import numpy as np #판다스 친구 넘파이
import pandas as pd # 판다스
from pandas import DataFrame as df
import re # 정규식 사용을 위함
import nltk
import csv
from konlpy.tag import Okt
from konlpy.utils import pprint
from collections import Counter

In [5]:
import os
dataset = pd.read_csv("total_dataset_trim.csv",encoding='utf-8')
dataset.head()

,NO,img_fold_no,studio_name,studio_picture,imafe_file,category,region,tag,price,area,floor,default_person,parking,description
0,1099,27,홍대_체스#4,['https://s3.hourplace.co.kr/temp/images/2019/...,"['27_0_홍대_체스#4.jpg', '27_1_홍대_체스#4.jpg', '27_2...",원룸,서울,#1.5룸#나혼자산다#모던#분리형원룸#빌라#서교동#서울시#원룸#클래식#홍대,"50,000",30,2,6,0,홍대 메인 거리에 등록한 오래된 집을 리모델링 했습니다. (90년대 빌라)\r\n체...
1,939,254,파티와 촬영용 이태원 뷰 좋은 루프탑 빌라,['https://s3.hourplace.co.kr/temp/images/2019/...,"['254_0_파티와_촬영용_이태원_뷰_좋은_루프탑_빌라.jpg', '254_1_파...",주택,서울,#10명수용#남산타워#루프탑#벽난로#복고풍인테리어#자연광스튜디오#전망,"38,000",145,2,8,1,"44평 내부에 큰 거실과 키친, 음악작업용 스튜디오 룸, 생활하는 큰 방, 렌트용 ..."
2,952,267,파티와 촬영용 이태원 뷰 좋은 루프탑 빌라,['https://s3.hourplace.co.kr/temp/images/2019/...,"['267_0_파티와_촬영용_이태원_뷰_좋은_루프탑_빌라.jpg', '267_1_파...",주택,서울,#10명수용#남산타워#루프탑#벽난로#복고풍인테리어#자연광스튜디오#전망,"38,000",145,2,8,1,"44평 내부에 큰 거실과 키친, 음악작업용 스튜디오 룸, 생활하는 큰 방, 렌트용 ..."
3,252,251,깔끔하고 아늑한 신축 투룸아파트,['https://s3.hourplace.co.kr/temp/images/2019/...,"['251_0_깔끔하고_아늑한_신축_투룸아파트.jpg', '251_1_깔끔하고_아늑...",아파트,서울,#10평투룸오피스텔#따뜻한가정집#선유도신축아파트#선유도신축오피스텔#신혼부부아파트#아...,"50,000",33,11,4,1,선유도역 1분거리에 위치한 신축 투룸오피스텔입니다.\r\n11층 남향으로 낮에는 해...
4,478,477,"TV없이 아늑하게 꾸민 거실, 21평 부부 공간",['https://s3.hourplace.co.kr/temp/images/2019/...,"['477_0_TV없이_아늑하게_꾸민_거실,_21평_부부_공간.jpg', '477_...",아파트,경기,#12층#TV없는#분위기좋은#색다른#아늑한거실#예쁜거실#깨끗한#쇼파2개#북향의#해가안드는,"62,000",69,12,10,3,- 2인가구 부부가 사는 공간입니다.\r\n- 신축이라 쾌적하고 깔끔합니다.\r\n...


In [337]:
## Tag 다듬기
## 복합명사로 이루어진 태그는 잘라서 길이 2~7 사이의 단어만 추출
def getTagList(value):
    okt=Okt()
    parseTagList=[]
    tempTagList=[]
    tags=str(value).split('#') # 태그별로 자르기
    for tag in tags:
        if len(tag)>1:
            okt.nouns(tag)  # 1개 태그 내 복합 명사 자르기
            for j in okt.nouns(tag):
                if(len(j)>=2|len(j)<9):
                    tempTagList.append(j)
    tempTagList=list(set(tempTagList))
    parseTagList.append(tempTagList)
    return parseTagList

In [338]:
## description 다듬기
def getDescList(value):
    okt=Okt()
    descList=[]
    value=value.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    descList=[i for i in okt.nouns(value) if len(i)>1]
    descList=list(set(descList))
    return descList

In [342]:
## 코어 태그 추출
def getCoreTags(n):
    coreTagList=[]
    okt=Okt()    
    for i in range(n):
        coreTag=""
        descript=getDescList(dataset.description[i]) # 한글자 이상만 뽑음
        tags=getTagList(dataset.tag[i])  # tag를 한번 더 파싱
        for t in tags: 
            if t in descript: # 태그가 description 단어에도 있는지 확인
                coreTag+=t
        if len(tags[0])<1: # 태그 수가 적으면(=1개 이하) desc의 단어로 대체
            for desc in descript:
                coreTagList.append(desc)
        elif len(coreTag)>=3:  # description과 일치하는 tag가 충분하면(3개 이상) 일치하는 태그 그대로 사용
            for core in coretag:
                coreTagList.append(core)
        else:  # description과 일치하는 태그는 없지만 태그 수는 충분할 때 추출한 tag를 그대로 사용
            for tag in tags:
                coreTagList.append(tag)
    return coreTagList

In [343]:
%time CoreTagDate=getCoreTags(18)

Wall time: 1.49 s


In [344]:
CoreTagDate

[['홍대', '원룸', '서울시', '클래식', '서교동', '빌라', '분리', '모던'],
 ['자연', '전망', '고풍', '벽난로', '루프', '명수'],
 ['자연', '전망', '고풍', '벽난로', '루프', '명수'],
 ['신축', '선유도', '가정', '아파트', '평투룸'],
 ['북향', '쇼파', '분위기'],
 ['카페', '신혼집', '빌라'],
 ['반포', '아파트', '잠원역', '소형', '강남', '잠원동'],
 ['자취', '원룸', '가구', '광고', '여자'],
 ['원룸', '싱글', '라이프', '주거', '고시원', '천호역'],
 ['골드', '블루', '서울', '화이트', '모던'],
 ['아이방', '테라스', '복층', '대형', '평수'],
 ['아파트', '서현역'],
 ['화실'],
 ['평지', '화장실', '주차', '편리', '빈티', '가능', '차량', '공간', '강화도', '공사'],
 ['아파트', '정원', '모던', '아이방'],
 ['오디오', '가구', '층집', '가정', '레디', '접시', '루왁', '소품', '화이트', '모던'],
 ['차편', '동물', '층집', '여의도', '촬영', '당산', '아이', '근처', '베란다'],
 ['돌담', '자연', '층집', '사계절', '종로구', '주의', '아이', '소개', '유리']]

## wordcloud
    단어간 유사도

In [331]:
from gensim.models import word2vec

num_features = 300    # Word vector dimensionality                      
min_word_count = 10   # Minimum word count                        
num_workers = 2     # Number of threads to run in parallel
context = 4          # Context window size                                                                                    
downsampling = 1e-3  # Downsample setting for frequent words

model = gensim.models.Word2Vec(CoreTagDate, workers=num_workers, 
                          size=num_features, min_count = min_word_count,
                          window = context, sample = downsampling)

model_name = "TagSimilarity"
model.save(model_name)

In [326]:
# model.wv.vocab
model.wv.most_similar("원목",topn=5)


[('자연', 0.26361364126205444),
 ('느낌', 0.2331203818321228),
 ('대관', 0.2019938975572586),
 ('신축', 0.20067569613456726),
 ('우드', 0.1839878112077713)]

In [ ]:
{'홍대': <gensim.models.keyedvectors.Vocab at 0x26c82b1e788>,
 '원룸': <gensim.models.keyedvectors.Vocab at 0x26c82b1e588>,
 '클래식': <gensim.models.keyedvectors.Vocab at 0x26c82b1ec48>,
 '빌라': <gensim.models.keyedvectors.Vocab at 0x26c82b1e888>,
 '모던': <gensim.models.keyedvectors.Vocab at 0x26c82b1ed48>,
 '자연': <gensim.models.keyedvectors.Vocab at 0x26c82b1ea88>,
 '전망': <gensim.models.keyedvectors.Vocab at 0x26c82b1e108>,
 '벽난로': <gensim.models.keyedvectors.Vocab at 0x26c82b1e508>,
 '루프': <gensim.models.keyedvectors.Vocab at 0x26c82b1e048>,
 '신축': <gensim.models.keyedvectors.Vocab at 0x26c82b1ed88>,
 '가정': <gensim.models.keyedvectors.Vocab at 0x26c82b1ed08>,
 '아파트': <gensim.models.keyedvectors.Vocab at 0x26c82b1e4c8>,
 '분위기': <gensim.models.keyedvectors.Vocab at 0x26c82b552c8>,
 '카페': <gensim.models.keyedvectors.Vocab at 0x26c82b55908>,
 '신혼집': <gensim.models.keyedvectors.Vocab at 0x26c82b55148>,
 '강남': <gensim.models.keyedvectors.Vocab at 0x26c82b55208>,
 '자취': <gensim.models.keyedvectors.Vocab at 0x26c82b55888>,
 '가구': <gensim.models.keyedvectors.Vocab at 0x26c82b55408>,
 '광고': <gensim.models.keyedvectors.Vocab at 0x26c82b55188>,
 '서울': <gensim.models.keyedvectors.Vocab at 0x26c82b55cc8>,
 '화이트': <gensim.models.keyedvectors.Vocab at 0x26c82b55248>,
 '아이방': <gensim.models.keyedvectors.Vocab at 0x26c82b55948>,
 '테라스': <gensim.models.keyedvectors.Vocab at 0x26c82b55d88>,
 '복층': <gensim.models.keyedvectors.Vocab at 0x26c82b55448>,
 '대형': <gensim.models.keyedvectors.Vocab at 0x26c82b554c8>,
 '이': <gensim.models.keyedvectors.Vocab at 0x26c82b550c8>,
 '공': <gensim.models.keyedvectors.Vocab at 0x26c82b55348>,
 '간': <gensim.models.keyedvectors.Vocab at 0x26c82b556c8>,
 '화': <gensim.models.keyedvectors.Vocab at 0x26c82b557c8>,
 '실': <gensim.models.keyedvectors.Vocab at 0x26ce02041c8>,
 '빈티': <gensim.models.keyedvectors.Vocab at 0x26ce02042c8>,
 '가능': <gensim.models.keyedvectors.Vocab at 0x26ce0204548>,
 '공간': <gensim.models.keyedvectors.Vocab at 0x26c827970c8>,
 '정원': <gensim.models.keyedvectors.Vocab at 0x26c82797148>,
 '층집': <gensim.models.keyedvectors.Vocab at 0x26c82797688>,
 '소품': <gensim.models.keyedvectors.Vocab at 0x26c82797608>,
 '촬영': <gensim.models.keyedvectors.Vocab at 0x26c82797d48>,
 '아이': <gensim.models.keyedvectors.Vocab at 0x26c82797808>,
 '베란다': <gensim.models.keyedvectors.Vocab at 0x26c82797188>,
 '심플': <gensim.models.keyedvectors.Vocab at 0x26c82797fc8>,
 '공원': <gensim.models.keyedvectors.Vocab at 0x26c82797c88>,
 '우드': <gensim.models.keyedvectors.Vocab at 0x26c82797408>,
 '하우스': <gensim.models.keyedvectors.Vocab at 0x26c82797ec8>,
 '테리어': <gensim.models.keyedvectors.Vocab at 0x26c82797508>,
 '그레이': <gensim.models.keyedvectors.Vocab at 0x26c82797c08>,
 '주방': <gensim.models.keyedvectors.Vocab at 0x26c82797288>,
 '거실': <gensim.models.keyedvectors.Vocab at 0x26c82797888>,
 '원목': <gensim.models.keyedvectors.Vocab at 0x26c82797708>,
 '고급': <gensim.models.keyedvectors.Vocab at 0x26c82797088>,
 '일산': <gensim.models.keyedvectors.Vocab at 0x26c82797788>,
 '주택': <gensim.models.keyedvectors.Vocab at 0x26c827972c8>,
 '영화': <gensim.models.keyedvectors.Vocab at 0x26c82797948>,
 '작업실': <gensim.models.keyedvectors.Vocab at 0x26c827974c8>,
 '셀프': <gensim.models.keyedvectors.Vocab at 0x26c82797a08>,
 '조명': <gensim.models.keyedvectors.Vocab at 0x26c82797dc8>,
 '계단': <gensim.models.keyedvectors.Vocab at 0x26c82797a88>,
 '바다': <gensim.models.keyedvectors.Vocab at 0x26c82797b48>,
 '시골': <gensim.models.keyedvectors.Vocab at 0x26c82797308>,
 '마': <gensim.models.keyedvectors.Vocab at 0x26c82797448>,
 '주': <gensim.models.keyedvectors.Vocab at 0x26c82797048>,
 '시': <gensim.models.keyedvectors.Vocab at 0x26c82797988>,
 '대': <gensim.models.keyedvectors.Vocab at 0x26c82797908>,
 '사': <gensim.models.keyedvectors.Vocab at 0x26c82797f48>,
 '용': <gensim.models.keyedvectors.Vocab at 0x26c82797d88>,
 '정': <gensim.models.keyedvectors.Vocab at 0x26c82797e88>,
 '도': <gensim.models.keyedvectors.Vocab at 0x26c807ff888>,
 '위': <gensim.models.keyedvectors.Vocab at 0x26c807ff788>,
 '가': <gensim.models.keyedvectors.Vocab at 0x26cffe2e888>,
 '장': <gensim.models.keyedvectors.Vocab at 0x26cffe2e0c8>,
 '소': <gensim.models.keyedvectors.Vocab at 0x26cffe2e648>,
 '오': <gensim.models.keyedvectors.Vocab at 0x26cffe2e9c8>,
 '전': <gensim.models.keyedvectors.Vocab at 0x26cffe2e1c8>,
 '지': <gensim.models.keyedvectors.Vocab at 0x26cffeba7c8>,
 '파': <gensim.models.keyedvectors.Vocab at 0x26cffebac48>,
 '미니': <gensim.models.keyedvectors.Vocab at 0x26cffebae48>,
 '파티': <gensim.models.keyedvectors.Vocab at 0x26cffeba248>,
 '바베큐': <gensim.models.keyedvectors.Vocab at 0x26cffeba848>,
 '옥상': <gensim.models.keyedvectors.Vocab at 0x26cffeba908>,
 '마당': <gensim.models.keyedvectors.Vocab at 0x26cffebab08>,
 '나무': <gensim.models.keyedvectors.Vocab at 0x26cff69b7c8>,
 '야외': <gensim.models.keyedvectors.Vocab at 0x26c828115c8>,
 '야경': <gensim.models.keyedvectors.Vocab at 0x26c82811388>,
 '갤러리': <gensim.models.keyedvectors.Vocab at 0x26c82811748>,
 '식물': <gensim.models.keyedvectors.Vocab at 0x26c82811588>,
 '영상': <gensim.models.keyedvectors.Vocab at 0x26cffafd308>,
 '한강': <gensim.models.keyedvectors.Vocab at 0x26c8288cdc8>,
 '서재': <gensim.models.keyedvectors.Vocab at 0x26c8288c148>,
 '부': <gensim.models.keyedvectors.Vocab at 0x26c8288ce88>,
 '아': <gensim.models.keyedvectors.Vocab at 0x26c8288c648>,
 '트': <gensim.models.keyedvectors.Vocab at 0x26cffe76408>,
 '고층': <gensim.models.keyedvectors.Vocab at 0x26cffe76fc8>,
 '구': <gensim.models.keyedvectors.Vocab at 0x26c82c26ec8>,
 '역': <gensim.models.keyedvectors.Vocab at 0x26c82c26bc8>,
 '제': <gensim.models.keyedvectors.Vocab at 0x26c82c26e88>,
 '촬': <gensim.models.keyedvectors.Vocab at 0x26c82c26048>,
 '영': <gensim.models.keyedvectors.Vocab at 0x26c82c26748>,
 '스': <gensim.models.keyedvectors.Vocab at 0x26c82c26988>,
 '방': <gensim.models.keyedvectors.Vocab at 0x26c82c26fc8>,
 '테': <gensim.models.keyedvectors.Vocab at 0x26c82c26d48>,
 '기': <gensim.models.keyedvectors.Vocab at 0x26c82c268c8>,
 '일': <gensim.models.keyedvectors.Vocab at 0x26c82c26948>,
 '인': <gensim.models.keyedvectors.Vocab at 0x26c82c26a48>,
 '조': <gensim.models.keyedvectors.Vocab at 0x26c82c26908>,
 '거': <gensim.models.keyedvectors.Vocab at 0x26cffc60d88>,
 '모임': <gensim.models.keyedvectors.Vocab at 0x26c827b1308>,
 '느낌': <gensim.models.keyedvectors.Vocab at 0x26c827b1448>,
 '가족': <gensim.models.keyedvectors.Vocab at 0x26c827b1a88>,
 '드라마': <gensim.models.keyedvectors.Vocab at 0x26c827b1688>,
 '대여': <gensim.models.keyedvectors.Vocab at 0x26c827b10c8>,
 '경기도': <gensim.models.keyedvectors.Vocab at 0x26c827b1b48>,
 '북유럽': <gensim.models.keyedvectors.Vocab at 0x26c827b1fc8>,
 '장소': <gensim.models.keyedvectors.Vocab at 0x26c827b1c48>,
 '리': <gensim.models.keyedvectors.Vocab at 0x26c827b19c8>,
 '사무실': <gensim.models.keyedvectors.Vocab at 0x26c827b1348>,
 '대관': <gensim.models.keyedvectors.Vocab at 0x26c827b1188>,
 '화보': <gensim.models.keyedvectors.Vocab at 0x26c827b1e08>,
 '개인': <gensim.models.keyedvectors.Vocab at 0x26c827b1048>,
 '사진': <gensim.models.keyedvectors.Vocab at 0x26c827b1608>,
 '감성': <gensim.models.keyedvectors.Vocab at 0x26c827b1888>,
 '렌탈': <gensim.models.keyedvectors.Vocab at 0x26cffe553c8>,
 '어': <gensim.models.keyedvectors.Vocab at 0x26cffe55488>,
 '원': <gensim.models.keyedvectors.Vocab at 0x26cffe55f08>,
 '연': <gensim.models.keyedvectors.Vocab at 0x26cffe55a08>,
 '잔디': <gensim.models.keyedvectors.Vocab at 0x26cffe55388>,
 '럭셔리': <gensim.models.keyedvectors.Vocab at 0x26cffe55d88>,
 '소규모': <gensim.models.keyedvectors.Vocab at 0x26cffe55f88>,
 '펜션': <gensim.models.keyedvectors.Vocab at 0x26c82b60e08>,
 '중': <gensim.models.keyedvectors.Vocab at 0x26c82b60948>,
 '유': <gensim.models.keyedvectors.Vocab at 0x26c82b60e88>,
 '치': <gensim.models.keyedvectors.Vocab at 0x26c82b603c8>,
 '디자인': <gensim.models.keyedvectors.Vocab at 0x26c82b60ac8>,
 '쇼핑몰': <gensim.models.keyedvectors.Vocab at 0x26c82b608c8>,
 '게스트하우스': <gensim.models.keyedvectors.Vocab at 0x26c82b60b08>,
 '채광': <gensim.models.keyedvectors.Vocab at 0x26c82b60f08>,
 '레트로': <gensim.models.keyedvectors.Vocab at 0x26c82b60c08>,
 '맛집': <gensim.models.keyedvectors.Vocab at 0x26c82b60848>,
 '수영장': <gensim.models.keyedvectors.Vocab at 0x26c82b60048>,
 '구조': <gensim.models.keyedvectors.Vocab at 0x26c82b60d08>,
 '연습실': <gensim.models.keyedvectors.Vocab at 0x26c82b60d88>,
 '호리': <gensim.models.keyedvectors.Vocab at 0x26c82b60a88>,
 '광렌탈': <gensim.models.keyedvectors.Vocab at 0x26c82b607c8>,
 '남산': <gensim.models.keyedvectors.Vocab at 0x26c82b60f48>}